In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from time import time
from os import listdir, path

from numpy.random import choice, shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from scipy.stats import zscore
from dtaidistance import dtw, dtw_c, dtw_ndim
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split


from IPython.display import clear_output

In [2]:
%load_ext autoreload
%autoreload 1

import preprocessing
import utils
from models import Encoder, DecoderLinear, DecoderLSTM, Sequence2Sequence

%aimport preprocessing
%aimport models

In [3]:
device = torch.device("cuda")
BASE_PATH = "../data/Univariate_arff"
TRAIN = "_TRAIN"
TEST = "_TEST"

BIG_CONST = 100000
datasets = [name for name in listdir(BASE_PATH) if path.isdir(path.join(BASE_PATH, name))]

In [4]:
with open("../data/length_Univariate_arff.json") as f:
    lengths = json.load(f)

In [ ]:
w = 3
k = 3
hidden_dim = 3
sample_size = 150

results = {name: {} for name in datasets}
try:
    for idx, problem in enumerate([name for name in datasets if lengths[name] < 200]):    
        length = lengths[problem]
        data_path = path.join(BASE_PATH, problem, problem)    
        pca = PCA(n_components=hidden_dim)
        X, y = preprocessing.get_dataset(data_path, length=length)

        X = X[:sample_size]
        y = y[:sample_size]
        train_set, test_set, valid_set = preprocessing.prepare_data(X, y, k, w, device)
        train_it = iter(train_set)
        for batch, _, _ in train_it:
            for x in batch.cpu().detach().numpy():
                pca = pca.fit(x)

        valid_it = iter(valid_set)
        batch, timeseries, labels = next(valid_it)
        batch = batch.cpu().detach().numpy()
        timeseries = timeseries.numpy()
        idxs = np.arange(len(timeseries)).reshape(-1, 1)

        scores_hidden = []
        scores_ts = []
        t = time()
        hiddens = np.stack([pca.transform(x) for x in batch])

        print(problem)
        scores_ts, scores_hidden = utils.classify(timeseries, hiddens, labels)
        results[problem]["ts"] = scores_ts
        results[problem]["hidden"] = scores_hidden
        results[problem]["shape"] = X.shape
        results[problem]["balance"] = np.unique(y, return_counts=True)
        print()

        with open("results.json", "w") as f:
            json.dump(lengths, f)

except Exception as exc:
    with open("results.json", "w") as f:
        json.dump(lengths, f)

    raise exc

# header = "{0:3s} | {1:^20s} | {2:^4s} | {3:^4s}".format("", "Problem", "Items", "Length")
# print(header)
# print("-"*len(header))   
# print("{0:3d} | {1:>20s} | {2:5d} | {3:6d}".format(
#     idx, problem[:20], X.shape[0], X.shape[1]))

ACSF1
hidden_ts: 19.805
raw_ts: 32.107
Raw ts score: 1.000 +- 0.000
Hidden ts score: 1.000 +- 0.000

Adiac
hidden_ts: 271.242
